# Document Search with LlamaIndex

This example shows how to use the Python [LlamaIndex](https://docs.llamaindex.ai/en/stable/) library to run a text-generation request on open-source LLMs and embedding models using the OpenAI SDK, then augment that request using the text stored in a collection of local PDF documents.

### <u>Requirements</u>
1. As you will accessing the LLMs and embedding models through Vector AI Engineering's Kaleidoscope Service (Vector Inference + Autoscaling), you will need to request a KScope API Key:
- If using **VPN**:
  
  Visit [https://kscope.vectorinstitute.ai/](https://kscope.vectorinstitute.ai/) and select *Request API Key*.
  
- If running **without VPN**:

  Run the following command (replace ```<user_id>``` and ```<password>```) from **within the cluster** to obtain the API Key. The ```access_token``` in the output is your KScope API Key.
  ```bash
  curl -X POST -d "grant_type=password" -d "username=<user_id>" -d "password=<password>" https://kscope.vectorinstitute.ai/token
  ```
2. After obtaining the `.env` configurations, make sure to create the ```.kscope.env``` file in your home directory (```/h/<user_id>```) and set the following env variables:
- For local models through Kaleidoscope (KScope):
    ```bash
    export OPENAI_BASE_URL="https://kscope.vectorinstitute.ai/v1"
    export OPENAI_API_KEY=<kscope_api_key>
    ```
- For OpenAI models:
   ```bash
   export OPENAI_BASE_URL="https://api.openai.com/v1"
   export OPENAI_API_KEY=<openai_api_key>
   ```
3. (Optional) Upload some pdf files into the `source_documents` subfolder under this notebook. We have already provided some sample pdfs, but feel free to replace these with your own.

## Set up the RAG workflow environment

In [1]:
from getpass import getpass
import os
from pathlib import Path

from llama_index import ServiceContext, SimpleDirectoryReader, VectorStoreIndex
# from llama_index.embeddings.cohereai import CohereEmbedding
# from llama_index.llms import Cohere
# from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.llms import OpenAILike, ChatMessage
# from llama_index.llms import ChatMessage

#### Load config files

In [2]:
import sys
from pathlib import Path

# Add root folder of the rag_bootcamp repo to PYTHONPATH
current_dir = Path().resolve()
parent_dir = current_dir.parent
sys.path.insert(0, str(parent_dir))

In [3]:
from utils.load_secrets import load_env_file
load_env_file()

In [4]:
GENERATOR_BASE_URL = os.environ.get("OPENAI_BASE_URL")
EMBEDDING_BASE_URL = os.environ.get("OPENAI_BASE_URL")

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

#### Set up some helper functions

In [5]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

#### Make sure other necessary items are in place

In [6]:
try:
    os.environ["COHERE_API_KEY"] = open(Path.home() / ".cohere.key", "r").read().strip()
    os.environ["CO_API_KEY"] = open(Path.home() / ".cohere.key", "r").read().strip()
except Exception:
    print(f"ERROR: You must have a Cohere API key available in your home directory at ~/.cohere.key")

# Look for the source-materials folder and make sure there is at least 1 pdf file here
contains_pdf = False
directory_path = "./source_documents"
if not os.path.exists(directory_path):
    print(f"ERROR: The {directory_path} subfolder must exist under this notebook")
for filename in os.listdir(directory_path):
    contains_pdf = True if ".pdf" in filename else contains_pdf
if not contains_pdf:
    print(f"ERROR: The {directory_path} subfolder must contain at least one .pdf file")

#### Set LLM and embedding model

In [7]:
GENERATOR_MODEL_NAME = "Meta-Llama-3.1-8B-Instruct"
EMBEDDING_MODEL_NAME = "bge-multilingual-gemma2" # "text-embedding-3-small"

## Start with a basic generation request without RAG augmentation

Let's start by asking the Cohere LLM a difficult, domain-specific question we don't expect it to have an answer to. A simple question like "*What is the capital of France?*" is not a good question here, because that's basic knowledge that we expect the LLM to know.

Instead, we want to ask it a question that is very domain-specific that it won't know the answer to. A good example would an obscure detail buried deep within a company's annual report. For example:

"*How many Vector scholarships in AI were awarded in 2022?*"

In [8]:
query = "How many Vector scholarships in AI were awarded in 2022?"

## Now send the query to KScope

In [9]:
llm = OpenAILike(model=GENERATOR_MODEL_NAME, is_chat_model=True, api_base=GENERATOR_BASE_URL, api_key=OPENAI_API_KEY)
message = [
    ChatMessage(
        role="user",
        content=query
    )
]
result = llm.chat(message)
# result = llm.complete(query)
print(f"Result: \n\n{result}")

Result: 

assistant: I don't have access to real-time data or specific information about the number of Vector scholarships in AI awarded in 2022. For the most accurate and up-to-date information, I recommend checking directly with Vector Institute or their official website. They would have the most current details on their scholarship programs and the number of awards given in 2022.


Without additional information, Cohere is unable to answer the question correctly. **Vector in fact awarded 109 AI scholarships in 2022.** Fortunately, we do have that information available in Vector's 2021-22 Annual Report, which is available in the `source_documents` folder. Let's see how we can use RAG to augment our question with a document search and get the correct answer.

## Ingestion: Load and store the documents from source-materials

Start by reading in all the PDF files from `source_documents`.

In [10]:
# # Load the pdfs
# pdf_folder_path = "./source_documents"
# documents = SimpleDirectoryReader(pdf_folder_path).load_data()
# print(f"Number of source materials: {len(documents)}\n")

## Define an embeddings model

This embeddings model will convert the textual data from our PDF files into vector embeddings. These vector embeddings will later enable us to quickly find the chunk of text that most closely corresponds to our original query.

In [11]:
# embed_model = CohereEmbedding(
#     model_name="embed-english-v3.0",
#     input_type="search_query"
# )
# service_context = ServiceContext.from_defaults(
#     embed_model=embed_model,
#     llm=llm,
#     chunk_size=200
# )

## Storage: Store the documents in a vector database

In [12]:
# index = VectorStoreIndex.from_documents(documents, service_context=service_context, show_progress=True)

## Retrieval: Now do a search to retrieve the chunk of document text that most closely matches our original query

In [13]:
# search_query_retriever = index.as_retriever(service_context=service_context)
# search_query_retrieved_nodes = search_query_retriever.retrieve(query)
# print(f"Search query retriever found {len(search_query_retrieved_nodes)} results")
# print(f"First result example:\n{search_query_retrieved_nodes[0]}\n")

That first result doesn't look right, but it's close? Could it be that we got the result that we wanted from that retrieval, but the results came back out of order? Let's try using a reranker to check which of our results is a closest match.

## Reranking: Improve the ordering of the document chunks

In [14]:
# reranker = CohereRerank()
# query_engine = index.as_query_engine(
#     node_postprocessors = [reranker]
# )

## Final RAG-augmented query

In [15]:
# result = query_engine.query(query)
# print(f"Result: {result}\n\n")